In [112]:
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option("display.max_rows", 999)

In [113]:
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [114]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [115]:
# end_dateの1ヶ月前の年月を取得する
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
exit_customer["exit_date"] = None
for i in range(0 ,len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

exit_customer["年月"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(exit_uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
# 欠損値の除去
exit_uselog = exit_uselog.dropna(subset=["name"])
exit_uselog = exit_uselog.reset_index()
# print(exit_uselog["customer_id"].unique())
exit_uselog.head()

# これで、退会したユーザーのデータを整形できた

,index,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
1,57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
2,110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
3,128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
4,147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,1.0,ナイト,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [117]:
# TODO: ノック43
# 継続ユーザーのデータを作成・整形する
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on="customer_id", how="left")
conti_uselog = conti_uselog.dropna(subset=["name"])
conti_uselog.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,0.0,ナイト,6000.0,通常,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201805,AS009373,4,3.0,XX,C01,F,2015-11-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
2,201805,AS015233,7,NaN,XXXXX,C01,M,2018-05-13,NaN,CA2,0.0,オールタイム,10500.0,入会費半額,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
3,201805,AS015315,3,6.0,XXXXX,C01,M,2015-07-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
4,201805,AS015739,5,7.0,XXXXX,C03,M,2017-06-01,NaN,CA1,0.0,ナイト,6000.0,通常,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0


In [118]:
# 継続顧客の重複を省く処理をする
# データをシャッフルし、顧客の重複を取り除く
# sample()・・・ランダムに一件データ取得する
# sample(frac=n) n*100%の割合でデータを取得する
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
conti_uselog.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,IK391344,6,5.0,XX,C01,M,2016-09-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.750000,5.0,7.0,3.0,1.0,2019-04-30,31.0
1,201805,GD009242,5,7.0,XXX,C02,F,2015-07-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.750000,5.0,7.0,2.0,1.0,2019-04-30,45.0
2,201903,TS578667,6,4.0,XXXXXX,C02,F,2017-05-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.916667,5.5,8.0,2.0,1.0,2019-04-30,23.0
3,201901,PL758737,6,7.0,XX,C01,M,2017-07-01,NaN,CA2,0.0,オールタイム,10500.0,入会費半額,6.000000,6.5,8.0,2.0,1.0,2019-04-30,21.0
4,201810,PL082485,6,7.0,XXXXX,C01,F,2017-12-01,NaN,CA3,0.0,オールタイム,10500.0,入会費無料,6.583333,6.5,9.0,5.0,1.0,2019-04-30,16.0


In [119]:
# 退会顧客と継続顧客のデータを縦に結合する
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
predict_data.head()

# これで、退会顧客と継続顧客のデータを作成完了

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,index,exit_date
0,201805,IK391344,6,5.0,XX,C01,M,2016-09-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.750000,5.0,7.0,3.0,1.0,2019-04-30,31.0,NaN,NaN
1,201805,GD009242,5,7.0,XXX,C02,F,2015-07-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.750000,5.0,7.0,2.0,1.0,2019-04-30,45.0,NaN,NaN
2,201903,TS578667,6,4.0,XXXXXX,C02,F,2017-05-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.916667,5.5,8.0,2.0,1.0,2019-04-30,23.0,NaN,NaN
3,201901,PL758737,6,7.0,XX,C01,M,2017-07-01,NaN,CA2,0.0,オールタイム,10500.0,入会費半額,6.000000,6.5,8.0,2.0,1.0,2019-04-30,21.0,NaN,NaN
4,201810,PL082485,6,7.0,XXXXX,C01,F,2017-12-01,NaN,CA3,0.0,オールタイム,10500.0,入会費無料,6.583333,6.5,9.0,5.0,1.0,2019-04-30,16.0,NaN,NaN


In [120]:
# 入会年月から対象の年月までの期間（月）を求める（periodカラムにデータ追加）
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data.iloc[i]["now_date"], predict_data.iloc[i]["start_date"])
    predict_data["period"][i] = delta.years*12 + delta.months
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,index,exit_date,period,now_date
0,201805,IK391344,6,5.0,XX,C01,M,2016-09-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.750000,5.0,7.0,3.0,1.0,2019-04-30,31.0,NaN,NaN,20,2018-05-01
1,201805,GD009242,5,7.0,XXX,C02,F,2015-07-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.750000,5.0,7.0,2.0,1.0,2019-04-30,45.0,NaN,NaN,34,2018-05-01
2,201903,TS578667,6,4.0,XXXXXX,C02,F,2017-05-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.916667,5.5,8.0,2.0,1.0,2019-04-30,23.0,NaN,NaN,22,2019-03-01
3,201901,PL758737,6,7.0,XX,C01,M,2017-07-01,NaN,CA2,0.0,オールタイム,10500.0,入会費半額,6.000000,6.5,8.0,2.0,1.0,2019-04-30,21.0,NaN,NaN,18,2019-01-01
4,201810,PL082485,6,7.0,XXXXX,C01,F,2017-12-01,NaN,CA3,0.0,オールタイム,10500.0,入会費無料,6.583333,6.5,9.0,5.0,1.0,2019-04-30,16.0,NaN,NaN,10,2018-10-01


In [122]:
# 欠損値を削除する
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               246
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
index                2842
exit_date            2842
period                  0
now_date                0
dtype: int64

In [123]:
# end_dateとexit_dateの欠損は仕方ないので、count_1が欠損しているデータを削除する
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2648
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
index                2648
exit_date            2648
period                  0
now_date                0
dtype: int64

In [124]:
# 文字列型の変数を処理できるように整形する

# 目的変数・説明変数に使うカラムだけを抽出する
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,オールタイム,M,5.0,1.0,20,0.0
1,通常,デイタイム,F,7.0,1.0,34,0.0
2,通常,デイタイム,F,4.0,1.0,22,0.0
3,入会費半額,オールタイム,M,7.0,1.0,18,0.0
4,入会費無料,オールタイム,F,7.0,1.0,10,0.0


In [125]:
# ダミー変数化する
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,20,0.0,0,0,1,1,0,0,0,1
1,7.0,1.0,34,0.0,0,0,1,0,1,0,1,0
2,4.0,1.0,22,0.0,0,0,1,0,1,0,1,0
3,7.0,1.0,18,0.0,1,0,0,1,0,0,0,1
4,7.0,1.0,10,0.0,0,1,0,1,0,0,1,0


In [126]:
# gender_M, campaign_name_通常, class_name_ナイト, を削除する
# これは、三つのうち二つが0ならば、必然的にもう一つのカラムが1だと判断できるから必要ない

del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,20,0.0,0,0,1,0,0
1,7.0,1.0,34,0.0,0,0,0,1,1
2,4.0,1.0,22,0.0,0,0,0,1,1
3,7.0,1.0,18,0.0,1,0,1,0,0
4,7.0,1.0,10,0.0,0,1,1,0,1


In [ ]:
# これで、機械学習前のデータ加工が終わった
# TODO: ノック47